## Generating Training Sets for Stage 3 - 

Fixed: Diameter (0.4 m), Depth Increments (0.01 m)

Variables (Inputs) : CPTu Dataset, Minimum Required Bearing Capacity 

Output: Optimised Length - Minimum Length to Give Required Capcity

In [1]:
import torch
from Functions_CPTu_Cap_Copy import *
from itertools import product

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# If CUDA is available, print out the number of CUDA devices and their names
if cuda_available:
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPUs detected.")

CUDA available: True
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 2060


## Importing CPTu Data

In [2]:
import pandas as pd
import numpy as np
## 
# 
# CPTu_data = pd.read_csv('CPTu_train_set.csv', header=None, skiprows=1)
CPTu_data = pd.read_csv('CPTu_val_set.csv', header=None, skiprows=1)
# CPTu_data = pd.read_csv('CPTu_test_set.csv', header=None, skiprows=1)

print(np.shape(CPTu_data))
print(type(CPTu_data))
print(CPTu_data.shape[1] // 4)

(3081, 12)
<class 'pandas.core.frame.DataFrame'>
3


## Creating Length and Required Bearing Capacity Inputs

In [3]:
l = np.arange(0.1, 30.81, 0.1)
RBC = np.arange(50, 2200, 50)
# b = np.arange(0.4, 1.6, 0.2)

b = 0.8

print(np.shape(RBC))
# zip = list(product(b, RBC))
# print((zip[:50]))
# print(zip[10][1])
# print(len(zip))

(43,)


## Implementing Training Loop

In [4]:
print(np.shape(CPTu_data), np.shape(l), np.shape(b), np.shape(RBC))

print(type(CPTu_data))

CPTu_data_np = CPTu_data.to_numpy()
print(type(CPTu_data_np))

outputs = []
for i in range(len(RBC)):
    print(i)
    
    temp_outputs = []

    temp_RBC = RBC[i]

    temp_outputs.append(s3_spec_bear_cap(CPTu_data_np, l, b, temp_RBC))

    outputs.append(temp_outputs)

print(np.shape(outputs))
# print(outputs)

# print(spec_bear_cap(CPTu_data_np, l, b, 200))
# 
# print(np.shape(spec_bear_cap(CPTu_data_np, l, b, 200)))

(3081, 12) (308,) () (43,)
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
(43, 1, 3, 1, 3)


## Changing Data Structure

In [5]:
outputs_squeezed = np.squeeze(outputs, axis=1)

print(np.shape(outputs_squeezed))
print(outputs_squeezed[3])

(43, 3, 1, 3)
[[[2.00000000e+02 1.00000000e-01 2.45739152e+02]]

 [[2.00000000e+02 9.00000000e-01 2.07448913e+02]]

 [[2.00000000e+02 1.50000000e+00 2.31491695e+02]]]


## Altering CPTu structure

In [6]:
columns_to_drop = [4 * i + j for i in range((CPTu_data.shape[1] // 4)) for j in range(2)]

# Drop the columns
CPTu_data_dropped = CPTu_data.drop(CPTu_data.columns[columns_to_drop], axis=1)

# Flattening the remaining columns for each borehole group
flattened_CPTu_groups = []
for i in range(0, CPTu_data_dropped.shape[1], 2):  # Iterate over each pair of remaining columns
    group = CPTu_data_dropped.iloc[:, i:i+2].values
    flattened_group = np.concatenate(group.T)  # Flatten the group transposed to concatenate by columns
    flattened_CPTu_groups.append(flattened_group)

# Output the shape and type for confirmation
print(np.shape(flattened_CPTu_groups))  # Expected shape: (no_BH, 20) if original was 10 samples per column
print(type(flattened_CPTu_groups))
print(flattened_CPTu_groups[0])

(3, 6162)
<class 'list'>
[0.   0.21 0.4  ... 0.   0.   0.  ]


## Export to CSV

In [7]:
# Setup the columns for DataFrame
columns = ['RBC'] + [f'CPTu_{i}' for i in range(1, 6163)] + ['Optim_L']

# Calculate the total number of rows in the final DataFrame
total_rows = len(RBC) * (CPTu_data.shape[1] //4)
print(total_rows)

# Pre-allocate a numpy array for all data
data_array = np.empty((total_rows, len(columns)))

print(CPTu_data.shape[1] //4)

# Fill the data array
current_row = 0
for i in range(len(RBC)):
    print(i)
    for j in range(CPTu_data.shape[1] // 4):
        # Each row is Length + Flattened CPTU data + Output
        data_array[current_row, 0] = RBC[i]
        data_array[current_row, 1:6163] = flattened_CPTu_groups[j]
        data_array[current_row, 6163] = outputs_squeezed[i][j][0][1] 
        current_row += 1

# Create a DataFrame from the numpy array
df = pd.DataFrame(data_array, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('Stage_3_vald_merged_reduced.csv', index=False)
print("CSV file has been created with the shape:", df.shape)


129
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
CSV file has been created with the shape: (129, 6164)


In [7]:
## PLotting a graph with validating and testing data